In [18]:
key = 'WS77-XIQ4-KGS1'
import pandas as pd
import requests
import xmltodict
import matplotlib.pyplot as plt
from reverse_phone_lookup import reverse_lookup, test_numbers

In [7]:
disconnected_df = pd.read_csv("disconnected.csv")
disconnected_df.head()
disconnected_phone_df = disconnected_df.sample(n=100)
disconnected_phone_df_trunc = disconnected_phone_df[['PHYSICIAN_FIRST_NAME', 'PHYSICIAN_LAST_NAME','OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE', 'COMMENTS']]

In [11]:
connected_df = pd.read_csv("connected.csv")
connected_df.head()

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,PRESENT_EMPLOYMENT_UPDATED,COMMENTS,Source,WSLIVE_SOURCE,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED
0,106060649,AARON,DAVID,SEIMS,NaN,NaN,STE 500,1720 S ORANGE AVE,ORLANDO,FL,...,1,COMPLETE,C,UPDATED,07/12/2019,0010606064172032806,00106060644075401000,NaN,PDS,1
1,401060596,ADAM,BRIAN,WOODRUFF,NaN,NaN,NaN,4802 E JOHNSON AVE,JONESBORO,AR,...,2,COMPLETE,C,UPDATED,07/12/2019,0040106059480272401,00401060598709368000,NaN,NEP,1
2,506891124,AARON,NaN,SPINGARN,NaN,NaN,WESTCHESTER E N T STE 100,141 S CENTRAL AVE,HARTSDALE,NY,...,1,COMPLETE,C,UPDATED,07/12/2019,005068911214110530,00506891129146863950,NaN,OTO,1
3,515880062,ALAN,CLARK,BECKMAN,NaN,NaN,NaN,1700 NICHOLASVILLE RD,LEXINGTON,KY,...,2,COMPLETE,C,UPDATED,07/12/2019,0051588006170040503,00515880068592606460,NaN,RO,1
4,577080381,ALEXANDRE,NaN,EACCARINO,NaN,NaN,NaN,2500 HOSPITAL BLVD STE 290,ROSWELL,GA,...,1,COMPLETE,C,UPDATED,07/12/2019,0057708038250030076,00577080384044335650,NaN,OBG,1


In [50]:
connected_phone_df = connected_df.sample(n =100)
connected_phone_df_trunc = connected_phone_df[['PHYSICIAN_FIRST_NAME', 'PHYSICIAN_LAST_NAME','OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE', 'COMMENTS']]
connected_phone_df_trunc.head()

,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_TELEPHONE,COMMENTS
1822,SHRUTI,KAPOOR,5100 W BROAD ST,COLUMBUS,OH,43228,6147884440,COMPLETE
2064,THOMAS,JANSSEN,9600 BROADWAY EXT,OKLAHOMA CITY,OK,73114,4052309435,COMPLETE
1994,PAYAM,YASHAR,8635 W 3RD ST STE 265W,LOS ANGELES,CA,90048,3105562020,COMPLETE
334,MACKENZIE,BARTZ,2755 S HIGHWAY 14 STE 2200,GREER,SC,29650,8648499555,COMPLETE
447,ANN,BLACK,1501 HOUSTON ST,CASTROVILLE,TX,78009,8305383550,COMPLETE


In [35]:
def reverse_lookup(phone_number, column_name, apikey):
    info_dict = {}
    base_url = "https://trial.serviceobjects.com/GPPL2/api.svc/GetPhoneInfo"
    parameters = {'PhoneNumber':phone_number, 'LicenseKey': apikey}
    response =  requests.get(base_url, params=parameters)
    results = xmltodict.parse(response.content)
    try:
        phone_results = results["PhoneInfoResponse"]["PhoneInfo"]
    except:
        pass
    info_dict[column_name] = phone_number
    try:
        info_dict["Name"] = phone_results["Contacts"]['Contact']["Name"]
    except:
        info_dict["Name"] = 'None'
    try:
        info_dict["Address"] = phone_results["Contacts"]['Contact']["Address"]
    except:
        info_dict["Address"] = 'None'
    try:
        info_dict["Zipcode"] = phone_results["Contacts"]['Contact']["PostalCode"]
    except:
        info_dict["Zipcode"] = 'None'
    try:
        info_dict["City"] = phone_results["Contacts"]['Contact']["City"]
    except:
        info_dict["City"] = 'None'
    try:
        info_dict["Zipcode"] = phone_results["Contacts"]['Contact']["PostalCode"][0:5]
    except:
        info_dict["Zipcode"] = 'None'
    try:
        info_dict["State"] = phone_results["Contacts"]['Contact']["State"]
    except:
        info_dict["State"] = 'None'
    try:
        info_dict["PhoneType"] = phone_results["Contacts"]['Contact']["PhoneType"]
    except:
        info_dict["PhoneType"] = 'None'
    try:
        info_dict["QualityScore"] = phone_results["Contacts"]['Contact']["QualityScore"]
    except:
        info_dict["QualityScore"] = 'None'
    try:
        info_dict["Date"] = phone_results["DateOfPorting"]
    except:
        info_dict["Date"] = 'None'
    try:
        info_dict["Notes"] = phone_results["NoteDescriptions"]
    except:
        info_dict["Notes"] = 'None'
    try:
        info_dict["Provider"] = phone_results['Provider']['Name']
    except:
        info_dict["Provider"] = 'None'
    return(info_dict, results)

In [45]:
def test_numbers(dataframe, apikey):

    #Create two empty lists:
    results_dict_list = []
    fun_massive_list = []

    #Iterate through dataframe and call reverse_lookup on each number:
    for row in dataframe.itertuples():
        new_dict = {}
        phone = row.OFFICE_TELEPHONE
        try:
            new_dict, phone_results = reverse_lookup(phone, 'OFFICE_TELEPHONE', apikey)
            results_dict_list.append(new_dict)
            fun_massive_list.append(phone_results)
        except:
            pass

    #Save lists of dictionaries as dataframes:
    new_df = pd.DataFrame(results_dict_list)
    final_df = pd.merge(new_df, dataframe, on = 'OFFICE_TELEPHONE')

    #Return dataframes:
    return(fun_massive_list, final_df)

In [47]:
massive_list, final_df = test_numbers(connected_phone_df_trunc.head(), key)
final_df

,Address,City,Date,Name,Notes,OFFICE_TELEPHONE,PhoneType,Provider,QualityScore,State,Zipcode,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,COMMENTS
0,1111 AUGUSTA DR,HOUSTON,None,KELSEY SEYBOLD,IsMailable,7134423222,BUSINESS,SOUTHWESTERN BELL,LOW,TX,77057,AMY,CHEN,560 MEYERLAND PLAZA MALL,HOUSTON,TX,77096,COMPLETE
1,None,None,2015-03-25,UC HEALTH WEST CHESTER HOSP,IsPorted,5132987325,BUSINESS,CIN BELL ANY DIST OH,MED,None,N/A,MARISSA,RAMIREZ,7700 UNIVERSITY DR,WEST CHESTER,OH,45069,COMPLETE
2,None,CAMARILLO,2018-06-05,SPANISH HILLS MEDICAL GROUP OHARA ROBERT R MD,"IsConnected,IsPorted",8054453310,BUSINESS,PEERLESS NTWK - CA,HIGH,CA,93010,LYNN,PLUCHE,400 MOBIL AVE STE C2,CAMARILLO,CA,93010,COMPLETE
3,None,PHILADELPHIA,None,THE CHILDRENS HOSPITAL OF PHILADELPHIA,IsConnected,2155904446,BUSINESS,VERIZON PENNSYLVANIA,MED,PA,19108,ROBERT,DOMS,3401 CIVIC CENTER BLVD,PHILADELPHIA,PA,19104,COMPLETE
4,None,LONG BRANCH,None,MONMOUTH MED CN,IsConnected,7329235195,BUSINESS,VERIZON NEW JERSEY,MED,NJ,07740,SHANNON,URSU,300 2ND AVE,LONG BRANCH,NJ,07740,COMPLETE


In [26]:
info_dict2, results2 = reverse_lookup('5704745072', 'OFFICE_TELEPHONE', key)
info_dict2

{'OFFICE_TELEPHONE': '5704745072',
 'Name': 'GEISINGER',
 'Address': '35 S MOUNTAIN BLVD',
 'City': 'MOUNTAIN TOP',
 'State': 'PA',
 'PhoneType': 'BUSINESS',
 'QualityScore': 'LOW',
 'Date': '2015-11-23',
 'Notes': 'IsMailable,IsPorted',
 'Provider': 'CTSI, INC. - PA'}

'18707'

In [52]:
connected_massive_list, connected_final_df = test_numbers(connected_phone_df_trunc, key)
connected_final_df.head()

,Address,City,Date,Name,Notes,OFFICE_TELEPHONE,PhoneType,Provider,QualityScore,State,Zipcode,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,COMMENTS
0,None,None,2019-04-01,OHIOHEALTH,"IsConnected,IsPorted,IsPossiblePortableVOIP",6147884440,BUSINESS,LEVEL 3 COMM - OH,MED,None,N/A,SHRUTI,KAPOOR,5100 W BROAD ST,COLUMBUS,OH,43228,COMPLETE
1,None,OKLAHOMA CITY,2017-03-31,MCBRIDE ORTHOPE,"IsConnected,IsPorted",4052309435,BUSINESS,COX OKLAHOMA TELECO,MED,OK,73102,THOMAS,JANSSEN,9600 BROADWAY EXT,OKLAHOMA CITY,OK,73114,COMPLETE
2,8635 W 3RD ST STE 265W,LOS ANGELES,2018-05-09,PAYAM ROBERT YASHAR MD INC PHYSICIANS GENERAL,"IsMailable,IsConnected,IsPorted,IsPossiblePort...",3105562020,BUSINESS,LEVEL 3 COMM - CA,HIGH,CA,90048,PAYAM,YASHAR,8635 W 3RD ST STE 265W,LOS ANGELES,CA,90048,COMPLETE
3,2755 S HIGHWAY 14 STE 2500,GREER,2008-12-23,MEDICAL GROUP OF THE CAROLINASDIVISION OF,"IsMailable,IsConnected,IsPorted",8648499555,BUSINESS,NUVOX COMMUNICATIONS,HIGH,SC,29650,MACKENZIE,BARTZ,2755 S HIGHWAY 14 STE 2200,GREER,SC,29650,COMPLETE
4,1501 HOUSTON ST,CASTROVILLE,2017-10-12,LITTLE ALSACE URGENT CARE CENTER CHILD CARE,"IsMailable,IsConnected,IsPorted",8305383550,BUSINESS,BANDWIDTH.COM - TX,HIGH,TX,78009,ANN,BLACK,1501 HOUSTON ST,CASTROVILLE,TX,78009,COMPLETE


In [53]:
disconnected_massive_list, disconnected_final_df = test_numbers(disconnected_phone_df_trunc, key)
disconnected_final_df.head()

,Address,City,Date,Name,Notes,OFFICE_TELEPHONE,PhoneType,Provider,QualityScore,State,Zipcode,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,COMMENTS
0,885 3RD AVE,NEW YORK,None,HQ GLOBAL WORKPLACES,IsMailable,2128295581,BUSINESS,VERIZON NEW YORK INC,LOW,NY,10022,ROBERT,TURNER,4341 US HIGHWAY 412 W,JACKSON,TN,38305.0,NOT IN SERVICE
1,885 3RD AVE,NEW YORK,None,HQ GLOBAL WORKPLACES,IsMailable,2128295581,BUSINESS,VERIZON NEW YORK INC,LOW,NY,10022,ROBERT,THORNTON,3701 LOOP RD,TUSCALOOSA,AL,35404.0,NOT IN SERVICE
2,885 3RD AVE,NEW YORK,None,HQ GLOBAL WORKPLACES,IsMailable,2128295581,BUSINESS,VERIZON NEW YORK INC,LOW,NY,10022,NIKHIL,SHAH,275 COLLIER RD NW STE 400,ATLANTA,GA,30309.0,NOT IN SERVICE
3,885 3RD AVE,NEW YORK,None,HQ GLOBAL WORKPLACES,IsMailable,2128295581,BUSINESS,VERIZON NEW YORK INC,LOW,NY,10022,MATEO,PORRES AGUILAR,7700 FLOYD CURL DR,SAN ANTONIO,TX,78229.0,NOT IN SERVICE
4,885 3RD AVE,NEW YORK,None,HQ GLOBAL WORKPLACES,IsMailable,2128295581,BUSINESS,VERIZON NEW YORK INC,LOW,NY,10022,SHIRISHA,BODANA,453 HESPER AVE,METAIRIE,LA,70005.0,NOT IN SERVICE


In [74]:
disconnected_final_df.to_csv('wtf.csv')
connected_final_df.to_csv('connected_final.csv')

In [73]:
disconnected_massive_list[10]

OrderedDict([('PhoneInfoResponse',
              OrderedDict([('@xmlns', 'http://www.serviceobjects.com'),
                           ('@xmlns:i',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('PhoneInfo',
                            OrderedDict([('Provider',
                                          OrderedDict([('Name',
                                                        'PACIFIC BELL'),
                                                       ('City', 'FONTANA'),
                                                       ('State', 'CALIFORNIA'),
                                                       ('Latitude', '34.0862'),
                                                       ('Longitude',
                                                        '-117.469'),
                                                       ('LineType',
                                                        'LANDLINE')])),
                                 

In [57]:
disconnected_phone_df_trunc

,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_TELEPHONE,COMMENTS
174,ROBERT,TURNER,4341 US HIGHWAY 412 W,JACKSON,TN,38305.0,2128295581,NOT IN SERVICE
173,ROBERT,THORNTON,3701 LOOP RD,TUSCALOOSA,AL,35404.0,2128295581,NOT IN SERVICE
227,STEN,SOLANDER,101 MANNING DR,CHAPEL HILL,NC,27514.0,9199661288,NOT IN SERVICE
13,AIZHANA,REPCHAK,3302 GALLOWS RD,FALLS CHURCH,VA,22042.0,2043957827,NOT IN SERVICE
2,ABNER,LANDRY,12551 INDIAN ROCKS RD STE 7,LARGO,FL,33774.0,7275860545,NOT IN SERVICE
275,WILLIAM,KIRKSEY,830 S GLOSTER ST,TUPELO,MS,38801.0,3852132546,NOT IN SERVICE
31,ANASTACIA,LAMBROU,1001 JOHNSON FERRY RD,ATLANTA,GA,30342.0,6036228963,NOT IN SERVICE
233,SUNIL,VALLURUPALLI,1505 NORTHSIDE BLVD STE 3000,CUMMING,GA,30041.0,7708860036,NOT IN SERVICE
228,STEPHEN,MASSICOTTE,3955 EAGLE CREEK PKWY STE C,INDIANAPOLIS,IN,46254.0,3172808410,NOT IN SERVICE
271,TAE,RHO,590 FARRINGTON HWY,KAPOLEI,HI,96707.0,8085219551,NOT IN SERVICE


In [59]:
disconnected_phone_df_trunc.iloc[45:90]

,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_TELEPHONE,COMMENTS
243,VELMA,CASANOVA,9850 KEY WEST AVE STE 100,ROCKVILLE,MD,20850.0,3017621818,NOT IN SERVICE
38,CHINYERE,NNADI,228 W ALEXANDER ST,PLANT CITY,FL,33563.0,8137542251,NOT IN SERVICE
236,SUSAN,WUTHRICH,200 MAIN ST STE 350,PAWTUCKET,RI,2860.0,4017267300,NOT IN SERVICE
10,ALLAN,YOUNG,165 N VILLAGE AVE STE 5,ROCKVILLE CTR,NY,11570.0,5164839020,NOT IN SERVICE
24,ADELITA,GREELEY,525 THIRD AVE,CHULA VISTA,CA,91910.0,8584992261,NOT IN SERVICE
54,DAVID,DUNCAN,3723 W 12600 S STE 150,RIVERTON,UT,84065.0,8014081980,NOT IN SERVICE
12,AHDY,BOLIS,5 MOBILE INFIRMARY CIR,MOBILE,AL,36607.0,2514357223,NOT IN SERVICE
254,ZIAULHAQ,ZIA,56 SCHOOLHOUSE RD STE B,WHITING,NJ,8759.0,7323507900,NOT IN SERVICE
57,MICHAEL,RAMSAY,169 ASHLEY AVE,CHARLESTON,SC,29425.0,8644556372,NOT IN SERVICE
259,WILLIAM,BASTNAGEL,1407 UNION AVE STE 620,MEMPHIS,TN,38104.0,9014486657,NOT IN SERVICE


174          TURNER
173        THORNTON
227        SOLANDER
13          REPCHAK
2            LANDRY
275         KIRKSEY
31          LAMBROU
233    VALLURUPALLI
228      MASSICOTTE
271             RHO
72          SHIBATA
253            HTAY
177        HATFIELD
182        PISINGER
47           PETERS
230        MARGOLIS
137             TOM
132          UPPALA
125         MAZUREK
205         VANGALA
214           REDDY
65      BROWN-BOLDS
121         MOLISON
165       ZWOLINSKI
225          NGUYEN
4            KAFAJI
16         LEADFORD
178           BLEZA
115         SHEPARD
46          HUFFMAN
           ...     
145         GUNDETI
189              OH
75       YOXTHEIMER
152       RODRIGUEZ
211        IKEJIANI
43             ZHOU
185           GUTTA
67       CUNNINGHAM
274        GRUNDLER
149            OBSU
150      ALYSKEWYCZ
234            GILL
209          LAMKIN
248          SUNEJA
120          THOMAS
59      PONNAMREDDY
199          BODANA
222            FORD
144            ARIF


KeyError: 'PhoneInfo'

NameError: name 'results4' is not defined

In [66]:
info_dict = {}
base_url = "https://trial.serviceobjects.com/GPPL2/api.svc/GetPhoneInfo"
parameters = {'PhoneNumber': '7848098180', 'LicenseKey': key}
response =  requests.get(base_url, params=parameters)
results4 = xmltodict.parse(response.content)

In [67]:
results4

OrderedDict([('PhoneInfoResponse',
              OrderedDict([('@xmlns', 'http://www.serviceobjects.com'),
                           ('@xmlns:i',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('Error',
                            OrderedDict([('Type', 'Domain Specific'),
                                         ('TypeCode', '4'),
                                         ('Desc',
                                          'Phone Number is not valid.'),
                                         ('DescCode', '1')]))]))])

In [71]:
if results4['PhoneInfoResponse']['Error']['Desc'] == 'Phone Number is not valid.'
    

'Phone Number is not valid.'

In [72]:
massive_list

[OrderedDict([('PhoneInfoResponse',
               OrderedDict([('@xmlns', 'http://www.serviceobjects.com'),
                            ('@xmlns:i',
                             'http://www.w3.org/2001/XMLSchema-instance'),
                            ('PhoneInfo',
                             OrderedDict([('Provider',
                                           OrderedDict([('Name',
                                                         'SOUTHWESTERN BELL'),
                                                        ('City', 'HOUSTON'),
                                                        ('State', 'TEXAS'),
                                                        ('Latitude',
                                                         '29.7652'),
                                                        ('Longitude',
                                                         '-95.4071'),
                                                        ('LineType',
                                  

In [75]:
disconnected_phone_df2 = disconnected_df.sample(n=10)

In [78]:
disconnected_phone_df2_trunc = disconnected_phone_df2[['PHYSICIAN_FIRST_NAME', 'PHYSICIAN_LAST_NAME','OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE', 'COMMENTS']]

In [79]:
disconnected_phone_df2_trunc

,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_TELEPHONE,COMMENTS
160,PEDRO,RAMOS,6 CALLE DR LOPEZ W,FAJARDO,PR,738.0,7878701529,NOT IN SERVICE
40,EDWARD,HOLSCHER,NaN,NaN,NaN,NaN,8033351219,NOT IN SERVICE
178,ROBERT,BLEZA,115 E 113TH AVE,CROWN POINT,IN,46307.0,2196638110,NOT IN SERVICE
191,ROBERTO,ALVAREZ-SWIHART,DEPT OF INTERN MEDICIN,AIBONITO,PR,705.0,7848098180,NOT IN SERVICE
109,MAREN,DUNN,18 MEADOW VILLAGE DR,BIG SKY,MT,59716.0,4069953111,NOT IN SERVICE
190,POLLIE,LUMBY,NaN,NaN,NaN,NaN,8889222732,NOT IN SERVICE
23,ABHAY,VAIDYA,8687 GRAND AVE,YUCCA VALLEY,CA,92284.0,7603651068,NOT IN SERVICE
103,KEVIN,URDAHL,307 WESTLAKE AVE N,SEATTLE,WA,98109.0,2062567293,NOT IN SERVICE
110,MARGARET,DIGERONIMO,7887 E BELLEVIEW AVE STE 1100,ENGLEWOOD,CO,80111.0,3034367142,NOT IN SERVICE
195,SCOTT,LIPOFF,4131 NW 13TH ST STE 101,GAINESVILLE,FL,32609.0,3523761887,NOT IN SERVICE


In [82]:
disconnected_df.drop_duplicates(subset='OFFICE_TELEPHONE',  inplace=True)

In [85]:
this = disconnected_df.sample(n=10)

In [86]:
this

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,PRESENT_EMPLOYMENT_UPDATED,COMMENTS,Source,WSLIVE_SOURCE,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED
126,3501900817,MICHAEL,JAMES,KLOTZ,NaN,NaN,NaN,125 RED CREEK DR STE 205,ROCHESTER,NY,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,035019008112514623,03501900815853210340,NaN,ORS,1
207,49565730168,TAMARISAKANDALA,NaN,NARAYAN,NaN,NaN,NaN,939 OFFICE PARK RD STE 308,W DES MOINES,IA,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,495657301693950265,49565730165152801252,NaN,CCA,1
135,1676880409,ROBERT,NORMAN,WHITE,NaN,NaN,NaN,800 BIESTERFIELD RD,ELK GROVE VLG,IL,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,016768804080060007,01676880408478273455,NaN,GP,1
133,1643821547,RICHARD,EATON,BESINGER,NaN,NaN,NaN,3700 SARATOGA AVE,DOWNERS GROVE,IL,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,0164382154370060515,01643821547082168563,NaN,MFM,1
97,3605970493,MARGARETE,SMITH,HOYLE,NaN,NaN,# 371,1959 N PEACE HAVEN RD,WINSTON SALEM,NC,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,0360597049195927106,03605970493362771600,NaN,PD,1
246,30801000041,VICTOR,A.,COLON SANCHEZ,NaN,NaN,OCEAN PARK,2069 CALLE BUENOS AIRES,SAN JUAN,PR,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,3080100004206900911,30801000047877283184,NaN,US,1
196,2507873033,SHARON,MARIE,RODDY,NaN,NaN,NaN,19111 W 10 MILE RD STE 164,SOUTHFIELD,MI,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,02507873031911148075,02507873032483540767,NaN,OBG,1
167,91706760011,RAZAULLAH,A,KHWAJA,NaN,NaN,NaN,311 MARLBORO RD,KENNETT SQ,PA,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,917067600131119348,91706760014106588420,NaN,HS,1
62,2401070233,RUCHIRA,MENKA,JHA,NaN,NaN,6TH FLOOR SCAIFE HALL,3550 TERRACE ST,PITTSBURGH,PA,...,1,NOT IN SERVICE,VT,OTHERS,07/12/2019,0240107023355015213,02401070236177262862,NaN,N,1
116,3305952019,MARTHA,CHRISTINA,KUTKO,NaN,NaN,WEILL CORNELL MEDICAL CENTER,525 E 68TH ST # 302,NEW YORK,NY,...,1,NOT IN SERVICE,C,OTHERS,07/12/2019,033059520152510065,03305952012019962403,NaN,CCP,1


In [87]:
massive_list3, final_df3 = test_numbers(this.head(), key)

In [90]:
final_df3[['PHYSICIAN_FIRST_NAME', 'PHYSICIAN_LAST_NAME','OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE', 'COMMENTS']]

,PHYSICIAN_FIRST_NAME,PHYSICIAN_LAST_NAME,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_TELEPHONE,COMMENTS
0,MICHAEL,KLOTZ,125 RED CREEK DR STE 205,ROCHESTER,NY,14623.0,5853210340,NOT IN SERVICE
1,TAMARISAKANDALA,NARAYAN,939 OFFICE PARK RD STE 308,W DES MOINES,IA,50265.0,5152801252,NOT IN SERVICE
2,ROBERT,WHITE,800 BIESTERFIELD RD,ELK GROVE VLG,IL,60007.0,8478273455,NOT IN SERVICE
3,RICHARD,BESINGER,3700 SARATOGA AVE,DOWNERS GROVE,IL,60515.0,7082168563,NOT IN SERVICE
4,MARGARETE,HOYLE,1959 N PEACE HAVEN RD,WINSTON SALEM,NC,27106.0,3362771600,NOT IN SERVICE
